In [ ]:
# Notebook che permette - interpolando un grafo stradale e un insieme di n poligoni - di estrarre n cluster, ognuno composto da un insieme di vie.

# Il notebook prende in input:
#     - il grafo stradale di un'area circoscritta prodotto in output dal notebook precedente (grafo_stradale_OSM_comune_filtered)
#     - file kml contenente un layer di poligoni che suddividono il grafo stradale in n aree (poligoni SCP final)

# Il notebook restituisce in output:
#     - il grafo stradale in formato geojson con aggiunta l'informazione del cluster di appartenenza per ogni via (grafo_stradale_OSM_comune_with_groups)
#     - un file csv con la suddivisione delle vie nei cluster (df_suddivisione_vie_per_cluster.csv)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import LineString, Polygon
import fiona
from keplergl import KeplerGl


# input

1) grafo stradale comune

In [ ]:
# grafo stradale da mappare che voglio dividere in cluster (sottopoligoni)
gdf_grafo_comune= gpd.read_file("data\grafo_stradale_OSM_comune_filtered.shp") 

In [ ]:
gdf_grafo_comune_proj = gdf_grafo_comune.to_crs(epsg=3003) # spaltial projection on Monte Mario / Italy zone 1
gdf_grafo_comune_proj ['lenght_km'] = gdf_grafo_comune_proj['geometry'].length/1000
print('lunghezza km vie da mappare in K : ',gdf_grafo_comune_proj['lenght_km'].sum()/10**3)

2) poligoni a cui assegnare le vie del grafo stradale così da suddividere le strade in sotto-gruppi (cluster)

In [ ]:
# poligoni per dividere la città di milano in gruppi di vie (sottografi)
fiona.supported_drivers['KML'] = 'rw'

gdf_poligon_sottografi = gpd.read_file(f'data\\poligoni SCP final.kml', driver='KML')
gdf_poligon_sottografi['sottografo_name'] = gdf_poligon_sottografi['Name']
gdf_poligon_sottografi.drop(['Name'],axis=1,inplace=True)
print('n poligoni/sottografi in cui dividere la città per estrarre le vie per ogni gruppo: ', len(gdf_poligon_sottografi))

In [ ]:
# controllo su presenza di duplicati
gdf_poligon_sottografi[gdf_poligon_sottografi[['sottografo_name']].duplicated()]

In [ ]:
# osservo un grafico che mostra il grafo stradale e il file poligonale 

In [ ]:
# gdf_grafo_comune_4326 = gdf_grafo_comune.to_crs(epsg=4326)
# gdf_poligon_sottografi_4326 = gdf_poligon_sottografi.to_crs(epsg=4326)
# kepler_map = KeplerGl(height=500,config={'config': {'mapStyle': {'styleType': 'Streets'}}})
# kepler_map.add_data(data=gdf_grafo_comune_4326,name='GRAFO')
# kepler_map.add_data(data=gdf_poligon_sottografi_4326[['geometry']],name='polygon data')
# kepler_map.config = {
#     'version': 'v1',
#     'config': {
#         'mapStyle': {
#             'styleType': 'light'
#         }
#     }
# }
# kepler_map


# main

a) creazione centroide per ogni poligono che serviranno poi agli step di post-processing del dato

In [ ]:
# creo centroidi con informazioni utili alle analisi successive
gdf_poligon_sottografi_info_centroid = gdf_poligon_sottografi.copy()
gdf_poligon_sottografi_info_centroid = gdf_poligon_sottografi_info_centroid.to_crs(epsg=3003) # spaltial projection on Monte Mario / Italy zone 1
gdf_poligon_sottografi_info_centroid['centroid_lat'] = gdf_poligon_sottografi_info_centroid['geometry'].centroid.y
gdf_poligon_sottografi_info_centroid['centroid_long'] = gdf_poligon_sottografi_info_centroid['geometry'].centroid.x
df_poligon_sottografi_info_centroid = gdf_poligon_sottografi_info_centroid[['sottografo_name','centroid_lat','centroid_long']]

gdf_poligon_sottografi_info_centroid = gpd.GeoDataFrame(
    df_poligon_sottografi_info_centroid, geometry=gpd.points_from_xy(df_poligon_sottografi_info_centroid.centroid_long, df_poligon_sottografi_info_centroid.centroid_lat), crs="3003"
)
gdf_poligon_sottografi_info_centroid.drop(['centroid_lat','centroid_long'],axis=1,inplace=True)
gdf_poligon_sottografi_info_centroid = gdf_poligon_sottografi_info_centroid.to_crs(epsg=4326)
gdf_poligon_sottografi_info_centroid['centroid_lat'] = gdf_poligon_sottografi_info_centroid['geometry'].y
gdf_poligon_sottografi_info_centroid['centroid_long'] = gdf_poligon_sottografi_info_centroid['geometry'].x
df_poligon_sottografi_info_centroid = gdf_poligon_sottografi_info_centroid[['sottografo_name','centroid_lat','centroid_long']]
df_poligon_sottografi_info_centroid.head(2)

In [ ]:
gdf_poligon_sottografi = gdf_poligon_sottografi.merge(df_poligon_sottografi_info_centroid,how='left')
gdf_poligon_proj = gdf_poligon_sottografi.to_crs(epsg=3003) # spaltial projection on Monte Mario / Italy zone 1
gdf_poligon_proj.head(2)

b) spatial join e estrazione sottografi/cluster 

In [ ]:
# si applica una regola tale per cui ogni via può essere asseganta a uno e a un solo poligono/cluster

In [ ]:
gdf_join =  gpd.sjoin(gdf_poligon_proj, gdf_grafo_comune_proj,predicate='intersects', how = 'left')#


print('lunghezza km vie da mappare in K : ',gdf_grafo_comune_proj['lenght_km'].sum()/10**3)
print('lunghezza km in K dopo spatial join prima di assegnare ogni via a un solo poligono (avendo quindi segmenti stradali duplicati): ',gdf_join['lenght_km'].sum()/(10**3))# maggiore 
# OSS 
# alcune vie cadono a cavallo fra poligoni 
# rimandendo ferreo alla regola tale per cui ogni via può essere asseganta a uno e a un solo poligono/cluster -->
# associo link di viee che cadono in più cluster al poligono in cui cadono in maniera preponderante

# assegno ogni via a un solo poligono (gruppo) (se una via cade in più poligoni la assegno al poligono in cui cade la porzione maggiore)
df_poligon_per_street_pre = gdf_join.groupby(['name','sottografo_name','centroid_lat','centroid_long'])['lenght_km'].sum().reset_index()
df_poligon_per_street = df_poligon_per_street_pre.loc[df_poligon_per_street_pre.groupby('name')['lenght_km'].idxmax()].reset_index(drop=True).drop('lenght_km',axis = 1)
df_poligon_per_street.head(2)

In [ ]:
# riassegno il gruppo di riferimento di ogni via al grafo stradale di open street map di partenza tramite una inner join sul "name" della via
# andando in inner join non ho i NAN dettati dal fatto che alcuni link stradali (osm_id) sono senza name della via
# aggiungo inoltre un filtro sugli osm_id di gdf_join overo quelli che cadono all'interno dei poligoni in cui va suddivisa la città

gdf_grafo_comune_proj_filtered = gdf_grafo_comune_proj[gdf_grafo_comune_proj['osm_id'].isin(gdf_join['osm_id'].unique())]
gdf_grafo_comune_with_group = gpd.GeoDataFrame(df_poligon_per_street[['name','sottografo_name']].merge(gdf_grafo_comune_proj_filtered),crs=3003)
gdf_grafo_comune_with_group = gdf_grafo_comune_with_group.to_crs(epsg=4326)

gdf_grafo_comune_with_group.head(2)

In [ ]:
print('lunghezza km vie da mappare in K : ',gdf_grafo_comune_proj['lenght_km'].sum()/10**3)
print('lunghezza km in K vie dopo spatial join prima di assegnare ogni via a un solo poligono (avendo quindi segmenti stradali duplicati): ',gdf_join['lenght_km'].sum()/(10**3))# 
print('lunghezza km in K vie dopo assegnazione di ogni via a un solo poligono e rimozione dei link stradali senza name della via: ',gdf_grafo_comune_with_group['lenght_km'].sum()/(10**3))# 

In [ ]:
# creo il dataframe df_vie_multiple_sottografi e il geodataframe gdf_vie_multiple_sottografi 
# che saranno poi utili allo step di post-processing del grafo
df_vie_multiple_sottografi = gdf_join[['name','sottografo_name']].drop_duplicates().groupby(['name']).size().reset_index().rename(columns={0: "n_zone_per_via"}).sort_values(by='n_zone_per_via',ascending=False).reset_index(drop=True)
df_vie_multiple_sottografi = df_vie_multiple_sottografi[df_vie_multiple_sottografi['n_zone_per_via']>1]

gdf_vie_multiple_sottografi = gdf_grafo_comune_proj[gdf_grafo_comune_proj['name'].isin(df_vie_multiple_sottografi['name'])][['osm_id','name','geometry']].reset_index(drop=True)
gdf_vie_multiple_sottografi['linestring']  = gdf_vie_multiple_sottografi['geometry'] 
gdf_vie_multiple_sottografi = gdf_vie_multiple_sottografi.to_crs(epsg = 3003)
gdf_vie_multiple_sottografi['geometry'] = gdf_vie_multiple_sottografi['geometry'].centroid
gdf_vie_multiple_sottografi = gdf_vie_multiple_sottografi.to_crs(epsg = 4326)
gdf_vie_multiple_sottografi['long'] = gdf_vie_multiple_sottografi['geometry'].x
gdf_vie_multiple_sottografi['lat'] = gdf_vie_multiple_sottografi['geometry'].y
gdf_vie_multiple_sottografi.head(2)

c) post-processing 

In [ ]:
# post-processing dettato dal fatto che potrebbero esistere vie con il medesimo nome, ma fra loro diverse, nella stessa città

# un esempio sulla città di milano è Via Eugenio Curiel che è presente ai due indirizzi 
# 20143 Milano MI:  45.4388516932489, 9.160468724045911
# Città metropolitana di Milano: 45.394352240709274, 9.172869750720498

# un altro esempio è Via Marco Polo che è sia a corsico appena fuori dal confine comunale di milano oltre giambellino ma anche a milano centro 20124 Milano MI


# al fine di evitare che segmenti stradali fra loro molto distanti cadano nel medesimo cluster (rendendone quasi impossibile la mappatura da parte di un gruppo)
# viene applicato un post processing per rimuovere una delle due "vie", quella più distante, dal cluster 
# si controlla quindi che segmenti della medesima via non siano fra loro troppo distanti.
# come distanza massima tollerata fra centroide cluster e segmenti della via uso 5.3 km
# in caso la distanza sia maggiore rimuovo quel segmento stradale.

# Accortezza dettata dal fatto che possono esserci strade diverse con il medesimo nome nella medesima città

In [ ]:
from math import radians, sin, cos, sqrt, atan2
def estimate_distance_between_point(lat1, lon1, lat2, lon2):
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    # Calculate the change in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    
    return distance


In [ ]:
# geodataframe contente i poligoni in cui vogliamo suddividere in grafo stradale totale della città cosi da ceare n sottocluster
gdf_poligon_sottografi.head(2)

In [ ]:
# dataframe contenente tutte le vie che cadono in almeno due cluster (poligoni) con il numero di poligoni (n_zone_per_via) in cui cade almeno un segmento della via
df_vie_multiple_sottografi.head(2) 


In [ ]:
# geodataframe contente i segmenti stradali delle vie che cadono in due cluster(poligoni) diversi
gdf_vie_multiple_sottografi.head(2)

In [ ]:
double_grafi_vie_list = gdf_vie_multiple_sottografi['name'].unique() # lista nomi viee che cadono in almeno due poligoni

osm_id_to_remove = []# lista in cui metterò gli osm_id da rimuovere perchè relativi a viee con nomi che appaiono in almeno due cluster diversi ma fra loro molto lontani
# si sceglie di rimuovere gli osm_id che cadono a una distanza maggiore di 5.3 km dal centroide del cluster di appartenza della via

for g in gdf_grafo_comune_with_group['sottografo_name'].unique():
    # ciclo sui sottografi
    gdf_g = gdf_grafo_comune_with_group[gdf_grafo_comune_with_group['sottografo_name']==g]
    df_vie_g_double = gdf_g[gdf_g['name'].isin(double_grafi_vie_list)]
    # per ogni sottografo controllo se ci sono viee che cadevano almeno in parte anche al di fuori di quel sottografo
    if len(df_vie_g_double)>0:
        g_centroid = gdf_poligon_sottografi[gdf_poligon_sottografi['sottografo_name']==g][['centroid_lat','centroid_long']].reset_index(drop=True)
        g_centroid_lat = g_centroid['centroid_lat']
        g_centroid_long = g_centroid['centroid_long']
        osm_id_g_list = df_vie_g_double['osm_id'].unique()

        for osm_g in osm_id_g_list:
            # osservo tutti i link stradali delle sottovie individiate e controllo se cadono a più di 5 km dal centroide del cluster 
            df_osm_id_centroid_g = gdf_vie_multiple_sottografi[gdf_vie_multiple_sottografi['osm_id']== osm_g][['name','lat','long']].reset_index(drop=True)
            osm_id_centroid_lat = df_osm_id_centroid_g['lat'][0]
            osm_id_centroid_long = df_osm_id_centroid_g['long'][0]
            name_via_osm_g = df_osm_id_centroid_g['name'][0]

            dist_osm_id_g = estimate_distance_between_point(osm_id_centroid_lat,osm_id_centroid_long,g_centroid_lat,g_centroid_long)

            if dist_osm_id_g > 5.3:
                # se il link stradale cade a più di 5.3 km dal centroide del cluster allora lo rimuovo dal cluster 
                # print('removing osm_id',osm_g)
                osm_id_to_remove.append(osm_g)
                # print(df_vie_multiple_sottografi[df_vie_multiple_sottografi['name']==name_via_osm_g])



########
list_name_vie_con_rimozioni = gdf_grafo_comune_with_group[gdf_grafo_comune_with_group['osm_id'].isin(osm_id_to_remove)]['name'].unique()
list_name_vie_con_rimozioni_string = ' - '.join(list_name_vie_con_rimozioni)
print(f'le vie "doppie" che hanno visto la rimozione di alcuni osm_id sono in totale {len(list_name_vie_con_rimozioni)} e sono le seguenti {list_name_vie_con_rimozioni_string}')

In [ ]:
gdf_grafo_comune_with_group_cleaned = gdf_grafo_comune_with_group[~gdf_grafo_comune_with_group['osm_id'].isin(osm_id_to_remove)].reset_index(drop=True)

In [ ]:
print('lunghezza km vie da mappare in K in input totali: ',gdf_grafo_comune_proj['lenght_km'].sum()/10**3)
print('lunghezza km vie da mappare in input a nan : ',gdf_grafo_comune_proj[gdf_grafo_comune_proj['name'].isna()]['lenght_km'].sum()/10**3)
print('lunghezza km in K vie dopo spatial join prima di assegnare ogni via a un solo poligono (avendo quindi segmenti stradali duplicati): ',gdf_join['lenght_km'].sum()/(10**3))# 
print('lunghezza km in K vie dopo assegnazione di ogni via a un solo poligono e rimozione dei link stradali senza name della via: ',gdf_grafo_comune_with_group['lenght_km'].sum()/(10**3))# 
print('lunghezza km in K vie dopo assegnazione di ogni via a un solo poligono e rimozione dei link stradali senza name della via più step di post-processing: ',gdf_grafo_comune_with_group_cleaned['lenght_km'].sum()/(10**3))# 

# salvo output

In [ ]:
gdf_grafo_comune_with_group_cleaned.to_file("grafo_stradale_OSM_comune_with_groups", driver="GeoJSON")  

In [ ]:
df_split_vie_per_cluster = gdf_grafo_comune_with_group_cleaned.groupby(['sottografo_name','name'])['lenght_km'].sum().reset_index()
df_split_vie_per_cluster.to_csv('df_suddivisione_vie_per_cluster.csv',decimal=',',sep=';',index=False,encoding='utf-8-sig')

In [ ]:
# grafico per osservare l'output ottenuo che mostra il grafo stradale assieme al file poligonale in input con la suddivisione in cluster delle vie

In [ ]:
# gdf_grafo_comune_4326 = gdf_grafo_comune_with_group_cleaned.to_crs(epsg=4326)
# gdf_poligon_sottografi_4326 = gdf_poligon_sottografi.to_crs(epsg=4326)
# kepler_map = KeplerGl(height=500,config={'config': {'mapStyle': {'styleType': 'Streets'}}})
# kepler_map.add_data(data=gdf_grafo_comune_4326,name='GRAFO')
# kepler_map.add_data(data=gdf_poligon_sottografi_4326[['sottografo_name','geometry']],name='polygon data')
# kepler_map.config = {
#     'version': 'v1',
#     'config': {
#         'mapStyle': {
#             'styleType': 'light'
#         }
#     }
# }
# kepler_map